In [1]:
# Consider such code
def my_func():
    inner_functs = []
    for i in range(5):
        def inner():
            return f"Hi from inner, my index is {i}"
        inner_functs.append(inner)
    return inner_functs


In [2]:
# a bit unexpected - each function uses last value from i. Why is that?
inner_functs = my_func()
for i in range(5):
    print(inner_functs[i]())

# it's because inner function is a closure, it's returned as a function and additional variables that it needs to run from my_func scope
# my_func scope doesn't exist when inner is called, but it has this additional variable i that it can access
# variable i has only one value - last value from before my_func scope got destroyed
# Python is doing that by using `cell` and "double-hop" through cell to the actual value
# check docs: https://docs.python.org/3/c-api/cell.html 

# basically 
# cell = function + free variables

Hi from inner, my index is 4
Hi from inner, my index is 4
Hi from inner, my index is 4
Hi from inner, my index is 4
Hi from inner, my index is 4


In [3]:
# we can check free vars and a closure cell directly
fn = inner_functs[0]
fn.__code__.co_freevars, fn.__closure__

(('i',), (<cell at 0x1035ebaf0: int object at 0x1015f8de0>,))

In [4]:
fn.__code__.co_freevars[0], fn.__closure__

('i', (<cell at 0x1035ebaf0: int object at 0x1015f8de0>,))

In [5]:
help(fn.__code__)

Help on code object:

class code(object)
 |  code(argcount, posonlyargcount, kwonlyargcount, nlocals, stacksize, flags, codestring, constants, names, varnames, filename, name, qualname, firstlineno, linetable, exceptiontable, freevars=(), cellvars=(), /)
 |
 |  Create a code object.  Not for the faint of heart.
 |
 |  Methods defined here:
 |
 |  __eq__(self, value, /)
 |      Return self==value.
 |
 |  __ge__(self, value, /)
 |      Return self>=value.
 |
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |
 |  __gt__(self, value, /)
 |      Return self>value.
 |
 |  __hash__(self, /)
 |      Return hash(self).
 |
 |  __le__(self, value, /)
 |      Return self<=value.
 |
 |  __lt__(self, value, /)
 |      Return self<value.
 |
 |  __ne__(self, value, /)
 |      Return self!=value.
 |
 |  __repr__(self, /)
 |      Return repr(self).
 |
 |  __sizeof__(...)
 |      Size of object in memory, in bytes.
 |
 |  co_lines(...)
 |
 |  co_positions(...)
 |
 |  replace(self,

In [6]:
def counter():
    # a closure
    count = 0
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

In [7]:
fn = counter()

In [8]:
print(fn())

1


In [9]:
print(fn())

2


In [10]:
print(fn())

3


In [11]:
print(fn())

4


In [12]:
# closures do not share memory, even if they are generated by the same func definition
# used cells are different

f1 = counter()
f2 = counter()
print(f1())
print(f1())

1
2


In [13]:
print(f2())
print(f2())

1
2


In [14]:
def adder(n):
    def inner(x):
        return x + n
    return inner


In [15]:
# no shared scopes here
a1 = adder(1)
a2 = adder(2)
a3 = adder(3)

In [16]:
print(a1(10))

11


In [17]:
print(a2(20))

22


In [18]:
print(a3(30))

33


In [19]:
# the closure have different cells, but all of them point to the same value from outer scope - `n` in this case!

adders = []
for n in range(5):
    # n is a free variable, this is a closure
    adders.append(lambda x: x + n)

In [20]:
# now all the cells to the same value, last value of y

for y in range(5):
    print(adders[y](y))

4
5
6
7
8


In [21]:
def outer():
    x = [1, 2, 3]
    print("ID outer", hex(id(x)))
    def inner():
        x = [1, 2, 3]
        print("ID innder", hex(id(x)))
    return inner
        

In [22]:
fn = outer()

ID outer 0x103669c00


In [23]:
fn()

ID innder 0x103669c00


In [24]:
def outer():
    x = [1, 2, 3]
    print("ID outer", hex(id(x)))
    def inner():
        y = x
        print("ID innder", hex(id(y)))
    return inner

In [25]:
fn = outer()

ID outer 0x1035e6980


In [26]:
print(fn.__closure__)

fn()

(<cell at 0x10366c1f0: list object at 0x1035e6980>,)
ID innder 0x1035e6980
